# **2. Predicción de textos, Parte I: N-Grams**

<div>
<img src="assets/autocomplete.png" width="500" style=" display: block; margin-left: auto; margin-right: auto;"/>
</div>

En esta sección, construiremos un sistema de **predicción de textos** el cual sugiere la siguiente palabra mientras un usuario está escribiendo. Los predictores de texto se utilizan en teclados de teléfonos móviles, aplicaciones de búsqueda, sistemas de redacción de correo electrónico impulsados por IA, y más.

El objetivo principal de un sistema de predicción de textos es que, *dada alguna secuencia de palabras, este pueda predecir la siguiente palabra según probabilidades*.

#### ¿Cómo lo hacemos nosotros?
En Procesamiento del Lenguaje Natural (PNL), esto se puede resolver con un **modelo del lenguaje**. Un modelo de lenguaje aprende la distribución de palabras en el texto. Construiremos un modelo de lenguaje sencillo que aprenda cadenas comunes de dos o tres palabras.
***

## **Preparación de datos**
### Cargar el corpus
Para esta aplicación, usaremos un texto en inglés tomado del corpus COCA. 

<div class="alert alert-block alert-warning">
    Como antes, siéntete libre de usar (un texto) de tu propia lengua en su lugar.
</div>

In [ ]:
# Load our data
import util

# REPLACE WITH YOUR CORPUS DIRECTORY
corpus = util.load_raw_text(corpus_directory="../corpora/eng")
corpus[:1000]

### Preprocesamiento y Tokenización
Al igual que el corrector ortográfico, el siguiente paso es tokenizar el texto en palabras individuales. La única diferencia es que mantenemos la puntuación.

#### **Ejercicio 1**
Use la expresión regular proporcionada para tokenizar el texto, y devolver el resultado.

<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">return re.findall(word_or_punctuation_regex, text)</code></pre>
</details>

In [ ]:
import re

word_or_punctuation_regex = r"[\w|\']+|[\.|\,|\?|\!]"

def preprocess(text):
    text = util.strip_accents(text)
    text = text.lower()

    # TODO: Use the provided regex to tokenize the text, and return the result

tokens_filtered = preprocess(corpus)

print(len(tokens_filtered), "total tokens")
print(tokens_filtered[:200])

## **Modelado de N-Gram(as)**
Ahora, vamos a construir nuestro primer modelo. Para esto usaremos *n-gram(as)*.

Un propio n-grama es simplemente una secuencia de palabras *n* que aparece en nuestro texto. Por ejemplo, considera la frase:

Cuál es tu Nombre? 
(n.t. con el fin de no modificar los ejemplos que siguen, se mantiene la convención ortográfica del inglés de usar solo un signo de cierre).

Si obtuviéramos la lista de todos los *bigramas* (2 palabras cada una) en la oración, tendríamos:

- *Cuál es*
- *es tu*
- *tu nombre*
- *Nombre ?*

Si obtuviéramos la lista de todos los *trigramas* (3 palabras cada una) en la oración, tendríamos:

- *Cuál es tu*
- *es tu nombre*
- *tu nombre* ?*

Y así pues, y así sucesivamente.

### Usando N-gramas como un modelo de lenguaje (natural)

Podemos utilizar fácilmente la idea de los propios n-gramas para construir un sencillo modelo de lenguaje natural. Por ejemplo, si estamos intentando predecir la siguiente palabra, usando trigrams, dada la entrada:

> Cuál es tu* ...

Podemos ver todos los trigrams que empiezan con *es tu*, y elegir el más común. Usemos nuestro texto tokenizado para obtener una lista de todos los n-gramas que aparecen en el texto. 

### Rellenando frases
Ahora mismo, podemos tener n-gramas en los que se cruzan los límites de las frases, tales como "titulares . tú". Esto no es muy útil, así que una técnica común es **rellenar** cada oración con tokens representando el inicio de la oración.

In [ ]:
def pad(text: list, num_padding: int):
    
    padded_text = []
    
    # Add initial padding to the first sentence
    for _ in range(num_padding):
        padded_text.append("<s>")
    
    for word in text:
        padded_text.append(word)

        # Every time we see an end punctuation mark, add <s> tokens after it
        # REPLACE IF YOUR LANGUAGE USES DIFFERENT END PUNCTUATION
        if word in [".", "?", "!"]:
            for _ in range(num_padding):
                padded_text.append("<s>")
        
        
    return padded_text

print(pad(tokens_filtered, 2)[:30])

### Crea una lista de n-gramas
El siguiente código utiliza la biblioteca **NLTK** para crear una lista de trigramas.

#### **Ejercicio 2**
¿Qué cambiaríamos para usar bigramas en su lugar?

<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">padded_tokens = pad(tokens_filtered, 1)
trigrams = list(ngrams(sequence=padded_tokens, n=2))</code></pre>
</details>

In [ ]:
from nltk.util import ngrams

# Now, we can actually create the list of n-grams using the NLTK library
padded_tokens = pad(tokens_filtered, 2)
trigrams = list(ngrams(sequence=padded_tokens, n=3))
trigrams[:30]

Ahora que tenemos una lista de triegramas, podemos contar la frecuencia de cada trigrama diferente.

#### **Ejercicio 3**
Utilizando la lista de trigramas, llena el diccionario `all_trigrams` de tal manera que cada llave es un trigrama único y el valor es cuántas veces dicho trigrama ocurre en el texto.

<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">for gram in trigrams:
    if gram in all_trigrams:
        all_trigrams[gram] += 1
    else:
        all_trigrams[gram] = 1</code></pre>
</details>

In [ ]:
# A dict of all trigrams and their frequency
all_trigrams = dict()

# TODO: Add each unique trigram to the dictionary and set the value to how many times that trigram occurs in the text
        
len(all_trigrams)

In [ ]:
# Let's see what the twenty most common trigrams are
sorted(all_trigrams.items(), key=lambda x: x[1], reverse=True)[:30]

## Haciendo predicciones usando n-gramas

Ahora que tenemos un recuento de todos los trigramas, podemos hacer predicciones buscando el trigrama más común que coincida con nuestro input. 

In [ ]:
def predict_trigram_model(text, number_results = 3):
    input_tokens = pad(preprocess(text), 2)
    
    # Find the last 2 tokens in the input
    last_two_tokens = input_tokens[-2:]
    
    # Search our list of all trigrams to find matching trigrams
    matching_trigrams = []
    for item in all_trigrams.items():
        gram = item[0]
        
        # Check if the first and second item in the trigram are a match
        if gram[0] == last_two_tokens[0] and gram[1] == last_two_tokens[1]:
            matching_trigrams.append(item)
    
    # Now, sort the matching trigrams by popularity and return the first `number_results` results
    sorted_matching_trigrams = sorted(matching_trigrams, key=lambda x: x[1], reverse=True)
    top_matching_trigrams = sorted_matching_trigrams[:number_results]
    
    # Last, let's just get the predicted word (the last word of the trigram)
    predictions = [trigram[0][2] for trigram in top_matching_trigrams]
    return predictions

predict_trigram_model("how is")

## Hacer una aplicación independiente de predicción de textos

#### **Ejercicio 4**
Crea una interfaz de usuario usando la función `predict_trigram_model`. Consulte el ejercicio del corrector ortográfico para obtener ayuda.

<details>
  <summary>Show answer</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">autocompleter = gr.Interface(fn=predict_trigram_model, inputs="text", outputs="text", live=True)
</code></pre>
</details>

In [ ]:
import gradio as gr

autocompleter = None

# TODO: Create a UI using Gradio for predictive text

autocompleter.launch()

# **Mejorando N-Gramas con Backoff**
Nuestra aplicación tiene un problema importante. Si la frase que ingresamos termina con dos palabras que no coinciden con *ninguna* trigrama, entonces nuestro modelo no tiene predicciones que hacer.

Una solución común para esto es utilizar un proceso llamado **backoff**. Por medio del 'backoff', si nuestro modelo de trigramas no encuentra ningún resultado, intentamos encontrar *bigramas* coincidentes en su lugar. Si eso falla, vamos a *unigramas* (lo que significa que sólo usamos las palabras más frecuentes). Esto significa que siempre conseguimos un resultado, aunque no sea tan bueno.

A continuación se muestra un modelo que utiliza backoff y un n-grama de tamaño arbitrario para hacer predicciones, juntando todo lo visto hasta ahora.

In [ ]:
n_gram_models = dict()

def create_ngram_model(n = 3):
    padded_tokens = pad(tokens_filtered, n - 1)
    grams = list(ngrams(sequence=padded_tokens, n=n))
    
    all_ngrams = dict()
    for gram in grams:
        if gram in all_ngrams:
            all_ngrams[gram] += 1
        else:
            all_ngrams[gram] = 1
    return all_ngrams


def predict_ngram_model(text, n = 3, number_results = 3):
    input_tokens = pad(preprocess(text), n - 1)
    
    while n > 0:
        # Find the last n - 1 tokens in the input
        last_tokens = tuple(input_tokens[-(n-1):])
    
        if not n in n_gram_models:
            n_gram_models[n] = create_ngram_model(n)
        
        matching_ngrams = []
        
        for item in n_gram_models[n].items():
            gram = item[0]
            if gram[:-1] == last_tokens:
                matching_ngrams.append(item)
    
        # Now, sort the matching n-grams by popularity and return the first `number_results` results
        sorted_matching_ngrams = sorted(matching_ngrams, key=lambda x: x[1], reverse=True)
        top_matching_ngrams = sorted_matching_ngrams[:number_results]
        
        # BACKOFF: If there are no results, drop n and try again
        if len(top_matching_ngrams) == 0:
            print("backing off to:", n-1)
            n = n - 1
            continue
    
        # Last, let's just get the predicted word (the last word of the trigram)
        predictions = [gram[0][-1] for gram in top_matching_ngrams]
        return predictions
    return []

predict_ngram_model("why is", 4)

**Resumen**
En este tutorial construimos una herramienta de predicción de texto para un lenguaje poco estudiado/documentado. Esto incluye:
- Crear una lista de n-gramas a partir de un corpus
- Utilizar n-gramas para predecir la siguiente palabra en una cadena de texto
- Usar 'backoff' para utilizar diferentes tamaños de n-gramas según sea necesario

### **Desafíos**
1. Utiliza la función de predicción repetidamente para predecir múltiples palabras secuenciales dada alguna cadena de texto. Por ejemplo, la cadena "Why is" podría ir seguida de "he doing" o "she haciendo".
2. Mejora la aplicación de texto predictivo con botones que te permiten añadir una sugerencia al texto que estás escribiendo.
3. Es un poco aburrido mostrar siempre las mejores predicciones. Añade un elemento de aleatoriedad utilizando la biblioteca `random` de Python, para que la función de predicción no muestre siempre los mismos resultados.